In [ ]:
import pandas as pd
import numpy as np
import warnings
import plotly.express as px
import plotly.graph_objects as go

warnings.filterwarnings('ignore')
np.random.seed(42)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.options.display.float_format = '{:,.2f}'.format

# Trabalhando os Dados Iniciais

Critérios de corte Inicial:
Importações menores que 1000 litros no total de 15 anos.
Países que só fizeram 1 ou 2 compras no período de 15 anos e não foram nos anos de 2021 ou 2022 (Compradores eventuais)
Corte da Rússia que causa uma distorção nos dados por conta de um acordo comercial entre Brasil e Rússia que fez o ano de 2009 ter um volume fora da realidade. Sem contar que a indefinição da guerra com a Ukrânia nao inspira confiança em investir em uma parceria neste momento.

In [ ]:
df = pd.read_excel('./ExportacaoVinhos.xlsx')
df_inicial = df[df.columns[0:31]]
df_inicial.head()

In [ ]:
df_inicial.shape

In [ ]:
df_melted = pd.melt(df_inicial, id_vars=['Pais'], var_name='Ano e Valor', value_name='Quantidade')
df_melted[['Ano', 'Valor']] = df_melted['Ano e Valor'].str.split('_', expand=True)
df_exportacao = df_melted.pivot(index=['Pais', 'Ano'], columns='Valor', values='Quantidade').reset_index()
df_exportacao.columns.name = None 
df_exportacao = df_exportacao.rename(columns={'KG': 'Litros', 'Pais' : 'Destino'})
df_exportacao['Origem'] = 'Brasil'
df_exportacao['Ano'] = pd.to_datetime(df_exportacao['Ano'], format='%Y')
df_exportacao['Litros'] = df_exportacao['Litros'].astype(float)
df_exportacao['USD'] = df_exportacao['USD'].astype(float)
df_exportacao.head()

In [ ]:
df_exportacao.shape

In [ ]:
df_exportacao.describe()

In [ ]:
df_exportacao.info()

In [ ]:
df_exportacao['Destino'].nunique()

#### Anotação: Nos últimos 15 anos nós exportamos vinho para 71 Países.

# Gerando a Tabela Solicitada Pelo Head de Dados

In [ ]:
df_total_por_pais = df_exportacao.groupby(['Origem','Destino']).sum(['Litros','USD']).sort_values('Litros', ascending=False)
df_total_por_pais.reset_index(inplace=True)
df_total_por_pais.head()

In [ ]:
df_total_por_pais.to_excel('./ExportacaoVinhosTotais.xlsx', index=False)

# Trabalhando com os dados totais dos últimos 15 anos

In [ ]:
df_total_por_pais['USD_por_Litro'] = df_total_por_pais['USD'] / df_total_por_pais['Litros']
df_total_por_pais.head(10)

### Optei por trabalhar com os dados dos top 10 em volume de exportação que representam 90,82% dos nossos dados.

In [ ]:
top10 = df_total_por_pais.sort_values('Litros', ascending=False)['Destino'].head(10).to_list()
top10

In [ ]:
df_total_top10 = df_total_por_pais[df_total_por_pais['Destino'].isin(top10)]
df_total_top10

In [ ]:
df_exportacao_top10 = df_exportacao[df_exportacao['Destino'].isin(top10)]
df_exportacao_top10.head()

In [ ]:

fig = px.bar(df_total_top10, x='Destino', y=['Litros'], title='Distribuição em Volume por Destino')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Países',
    xaxis_title_standoff=55,
    yaxis_title='Milhões de Litros',
    yaxis_type="log",
    yaxis_showticklabels=False,
    showlegend=False, 
    template="plotly_dark",
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)

fig.show()

In [ ]:
fig = px.bar(df_total_top10, x='Destino', y=['USD'], title='Distribuição em Dólares por Destino')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Países',
    xaxis_title_standoff=55,
    yaxis_title='Milhões de Dólares',
    yaxis_type="log",
    yaxis_showticklabels=False,
    showlegend=False, 
    template="plotly_dark",
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='US$ %{y:.2s}', textposition='outside', textfont_size=10)

fig.show()

In [ ]:
fig = px.bar(df_total_top10.sort_values(by='USD_por_Litro',ascending=False), x='Destino', y=['USD_por_Litro'], title='Valor em Dolar por Litro no Destino')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Países',
    xaxis_title_standoff=55,
    yaxis_title='Dólares',
    yaxis_type="log",
    yaxis_showticklabels=False,
    showlegend=False, 
    template="plotly_dark",
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='US$ %{y:.2s}', textposition='outside', textfont_size=10)

fig.show()

In [ ]:
df_producao_exportacao = pd.read_excel('./ProducaoExportacao.xlsx')
df_producao_exportacao['Ano'] = pd.to_datetime(df_producao_exportacao['Ano'], format='%Y')
df_producao_exportacao['Produzido'] = df_producao_exportacao['Produzido'].astype(float)
df_producao_exportacao['Exportado'] = df_producao_exportacao['Exportado'].astype(float)
df_producao_exportacao['PTAX'] = df_producao_exportacao['PTAX'].astype(float)
df_producao_exportacao.head()

In [ ]:
fig = px.bar(df_producao_exportacao.sort_values(by='Ano',ascending=False), x='Ano', y=['Produzido','Exportado'], title='Produzido vs Exportado', barmode='group')

fig.update_layout(
    title_font=dict(size=24), 
    xaxis_title='Anos',
    xaxis_title_standoff=55,
    yaxis_title='Milhões de Litros',
    yaxis_type="log",
    yaxis_showticklabels=False,
    template="plotly_dark",
    xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
     annotations=[
        dict(
            x=1,
            y=-0.32,
            xref='paper',
            yref='paper',
            text='* Escala Logarítmica',
            showarrow=False,
            font=dict(size=12)
        )
    ]
)

fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)



fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Produzido'], name='Produzido'))
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)
fig.add_trace(go.Scatter(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Produzido'], mode='lines', name='Tendência', line=dict(color='red')))

fig.update_layout(title='Variação da Produção Anual com Linha de Tendência',
                  barmode='group',
                  title_font=dict(size=24), 
                  xaxis_title='Anos',
                  xaxis_title_standoff=55,
                  yaxis_type="log",
                  yaxis_showticklabels=False,
                  showlegend=False, 
                  yaxis_title='Produção em Milhões de Litros',
                  template="plotly_dark",
                  xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
                  annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

fig.show()

#### Observação: Em 2016, a safra de uvas teve uma das maiores quebras registradas, com uma redução de 57% em relação ao ano anterior, equivalente, aproximadamente, a 300 milhões de quilos de uvas no território gaúcho. A principal causa da quebra histórica no ano 2016 foi uma série de acontecimentos climáticos que prejudicaram o desenvolvimento das uvas ao longo do ano, como geadas e excesso de chuvas.

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Exportado'], name='Exportado'))
fig.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)
fig.add_trace(go.Scatter(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['Exportado'], mode='lines', name='Tendência', line=dict(color='red')))
fig.update_layout(title='Variação da Exportação Anual com Linha de Tendência',
                  barmode='group',
                  title_font=dict(size=24), 
                  xaxis_title='Anos',
                  xaxis_title_standoff=55,
                  yaxis_type="log",
                  yaxis_showticklabels=False,
                  showlegend=False, 
                  yaxis_title='Exportação em Milhões de Litros',
                  template="plotly_dark",
                  xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
                  annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['PTAX'], name='PTAX'))
fig.update_traces(texttemplate='%{y:.4s}', textposition='outside', textfont_size=10)
fig.add_trace(go.Scatter(x=df_producao_exportacao['Ano'], y=df_producao_exportacao['PTAX'], mode='lines', name='Tendência', line=dict(color='red')))
fig.update_layout(title='Variação da PTAX Anual com Linha de Tendência',
                  yaxis_title='PTAX',
                  barmode='group',
                  title_font=dict(size=24), 
                  xaxis_title='Anos',
                  yaxis_type="log",
                  xaxis_title_standoff=55,
                  yaxis_showticklabels=False,
                  showlegend=False,
                  template="plotly_dark",
                  xaxis=dict(tickvals=df_producao_exportacao['Ano'], ticktext=df_producao_exportacao['Ano'].dt.year),
                  annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

fig.show()

In [ ]:
total_produzido = df_producao_exportacao['Produzido'].sum()
total_exportado = df_producao_exportacao['Exportado'].sum()
dolar_medio = df_producao_exportacao['PTAX'].mean()
percentual_exportado= (total_exportado/total_produzido) * 100

print(f"Total Produzido: {total_produzido}")
print(f"Total Exportado: {total_exportado}")
print(f"Percentual Exportado: {percentual_exportado}")
print(f"Dolar Médio: {dolar_medio}")

# Pegando dados de consumo, importação, exportação inflação projetada e produção do top 10

In [ ]:
df_dados_top10 = pd.read_excel('./top10.xlsx')
df_dados_top10['Ano'] = pd.to_datetime(df_dados_top10['Ano'], format='%Y')
df_dados_top10['Litros'] = df_dados_top10['Litros'].astype(float)
df_dados_top10.head(10)

In [ ]:
df_pivot = df_dados_top10.pivot_table(index=['Ano', 'Pais'], columns='Variavel', values='Litros', aggfunc='sum').reset_index()
df_pivot.fillna(0, inplace=True)
df_pivot['Saldo'] =  df_pivot['Produção'] + df_pivot['Importação'] - df_pivot['Exportação'] - df_pivot['Consumo']
df_pivot.head()

In [ ]:
df_economia_top10 = pd.read_excel('./WEO_Data.xlsx')
df_economia_top10['Ano'] = pd.to_datetime(df_economia_top10['Ano'], format='%Y')
df_pivot_economia = df_economia_top10.pivot_table(index=['Ano', 'Pais'], columns='Variavel', values='Percentual', aggfunc='sum').reset_index()
df_pivot_economia.fillna(0, inplace=True)
df_pivot_economia.head()

In [ ]:

figs = []

for pais in df_pivot['Pais'].unique():
    df_temp = df_pivot[df_pivot['Pais'] == pais]
    df_temp2 = df_exportacao_top10[df_exportacao_top10['Destino'] == pais]
    
    fig2 = px.bar(df_temp2, 
                  x='Ano', 
                  y='Litros', 
                  title=f'Brasil para {pais} por ano',
                  labels={'Litros': 'Litros Exportados', 'Ano': 'Ano'})
    
    fig2.update_layout(title=f'Brasil para {pais} por ano',
                      xaxis_title='Ano',
                      yaxis_title='Litros',
                      xaxis_tickangle=-45,
                      legend_title_text='',
                      yaxis_type="log",
                      xaxis_title_standoff=30,
                      title_font=dict(size=24), 
                      template="plotly_dark",
                      showlegend=False, 
                      yaxis_showticklabels=False,
                      xaxis=dict(tickvals=df_temp['Ano'], ticktext=df_temp['Ano'].dt.year),
                      annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])
    
    fig2.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)

    fig = px.bar(df_temp,
                 x="Ano",
                 y=["Consumo", "Importação", "Produção", "Exportação"],
                 color_discrete_sequence=["blue", "green", "red","orange"],
                 barmode="group")
    
    fig.update_layout(title=f'Dados - {pais}',
                      xaxis_title='Ano',
                      yaxis_title='Litros',
                      xaxis_tickangle=-45,
                      legend_title_text='',
                      yaxis_type="log",
                      xaxis_title_standoff=30,
                      title_font=dict(size=24), 
                      template="plotly_dark",
                      xaxis=dict(tickvals=df_temp['Ano'], ticktext=df_temp['Ano'].dt.year),
                      annotations=[
                        dict(
                            x=1,
                            y=-0.32,
                            xref='paper',
                            yref='paper',
                            text='* Escala Logarítmica',
                            showarrow=False,
                            font=dict(size=12)
                        )
                    ])

    fig1 = go.Figure()
    fig1.add_trace(go.Bar(x=df_temp['Ano'], y=df_temp['Saldo'], name='Saldo'))
    fig1.update_traces(texttemplate='%{y:.2s}', textposition='outside', textfont_size=10)
    fig1.add_trace(go.Scatter(x=df_temp['Ano'], y=df_temp['Saldo'], mode='lines', name='Tendência', line=dict(color='red')))
    
    fig1.update_layout(title=f'Saldo Anual com Linha de Tendência - {pais}',
                    xaxis_title='Ano',
                    yaxis_title='Litros',
                    legend_title_text='',
                    showlegend=False, 
                    title_font=dict(size=24),
                    xaxis_title_standoff=30,
                    template="plotly_dark",
                    xaxis_tickangle=-45,
                    xaxis=dict(tickvals=df_pivot['Ano'], ticktext=df_pivot['Ano'].dt.year),
                    )
    
    
    
    fig3 = go.Figure()
    subset = df_pivot_economia[df_pivot_economia['Pais'] == pais]
    fig3.add_trace(go.Scatter(x=subset['Ano'], y=subset['Inflação'],
                    mode='lines+markers',
                    name=pais))

    fig3.update_layout(title=f'Projeção de Inflação nos Próximos anos - {pais}',
                    xaxis_title='Ano',
                    yaxis_title='Inflação (%)',
                    title_font=dict(size=24),
                    xaxis_title_standoff=30,
                    template="plotly_dark",
                    xaxis=dict(tickvals=subset['Ano'], ticktext=subset['Ano'].dt.year))

    figs.append(fig)
    figs.append(fig1)
    figs.append(fig2)
    figs.append(fig3)

for fig in figs:
    fig.show()


